In [1]:
#importer les librairies
import pandas as pd
import re


# création de la fonction qui permet de récupérer les tables dans une requête sql

In [2]:
# La requête SQL de test
sql_query = """
SELECT Clients.Nom, Clients.Prenom, Clients.Email, SUM(Produits.Prix * DétailsCommande.Quantité) AS MontantTotal
FROM Clients
JOIN Commandes ON Clients.IdClient = Commandes.IdClient
JOIN DétailsCommande ON Commandes.IdCommande = DétailsCommande.IdCommande
JOIN Produits ON DétailsCommande.IdProduit = Produits.IdProduit
WHERE Commandes.DateCommande >= DATE_SUB(NOW(), INTERVAL 1 MONTH)
GROUP BY Clients.IdClient
HAVING MontantTotal >= 100;
"""

# Expression régulière pour extraire les noms de table
pattern = r'\b(?:FROM|JOIN)\s+(\w+)\b'

# Recherche des correspondances dans la requête SQL
table = re.findall(pattern, sql_query, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)


Tables utilisées dans la requête SQL:
['Clients', 'Commandes', 'DétailsCommande', 'Produits']


## Test avec différentes requêtes

In [3]:
sql1="select * from (select nom, adresse from Immeuble where id=1) as Koudalou"

table = re.findall(pattern, sql1, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)

Tables utilisées dans la requête SQL:
['Immeuble']


In [4]:
sql="SELECT * FROM Table"

table = re.findall(pattern, sql, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)

Tables utilisées dans la requête SQL:
['Table']


In [5]:
sql3="SELECT * FROM question WHERE q_id IN (SELECT r_fk_question_id FROM reponse WHERE r_date_ajout BETWEEN '2013-01-01' AND '2013-12-31')"

table = re.findall(pattern, sql3, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)

Tables utilisées dans la requête SQL:
['question', 'reponse']


In [6]:
sql4="SELECT sum(a.Population)FROM travail.communes64 AS a WHERE EXISTS(SELECT * FROM travail.communes64 AS b WHERE b.Population > 10)"

table = re.findall(pattern, sql4, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)

Tables utilisées dans la requête SQL:
['travail', 'travail']


# Création d'un dataframe pour faire tourner le script sur un fichier csv

In [7]:
#créer le dataframe à partir d'un fichier csv
df=pd.read_excel("ATP_requete.xlsx", header=None)

c:\Python\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
#lire toutes les infos du df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [9]:
#lire les 10 premières lignes du dataframe
df.head(10)

,0,1,2
0,NaN,tPostgresqlRow,truncate table job_referentiel
1,NaN,tPostgresqlRow,truncate table job_hierarchy
2,NaN,tPostgresqlRow,truncate table job_table
3,NaN,tPostgresqlRow,truncate table job_sql_table
4,NaN,tPostgresqlRow,truncate table job_load_context
5,NaN,tPostgresqlRow,truncate table job_variable
6,NaN,tPostgresqlRow,truncate table job_group_context
7,NaN,tPostgresqlRow,truncate table err_process_sans_properties
8,NaN,tPostgresqlRow,truncate table job_hierarchy_level
9,NaN,tPostgresqlRow,truncate table job_plan_tac


In [10]:
def extraire_table(sql_query):
    # Expression régulière pour extraire les noms de table
    pattern = r'\b(?:FROM|JOIN)\s+(\w+)\b'

    # Recherche des correspondances dans la requête SQL
    table = re.findall(pattern, sql_query, re.IGNORECASE)
    return table 

# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df['table'] = df[2].apply(extraire_table)

# S'assurer que la colonne 'table' est de type chaîne
df['table'] = df['table'].astype(str)

# Supprimer les crochets de la colonne 'table'
df['table']=df['table'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df['table'] = df['table'].str.replace('\'', '')

# Afficher le DataFrame résultant
df.tail(10)

,0,1,2,table
14,NaN,tPostgresqlInput,select * from job_hierarchy,job_hierarchy
15,NaN,tPostgresqlInput,"select t.generatedjobname,t.jobversion\r\nfrom job_plan_tac t\r\n",job_plan_tac
16,NaN,tPostgresqlInput,"SELECT \n talend_tracking.public.job_referentiel.projet, \n talend_tracking.public.job_referentiel.nom_fichier, \n talend_tracking.public.job_referentiel.nom_fichier_sans_extension, \n talend_tracking.public.job_referentiel.id_job, \n talend_tracking.public.job_referentiel.label_job, \n talend_tracking.public.job_referentiel.display_job,\r\n talend_tracking.public.job_referentiel.version\nFROM talend_tracking.public.job_referentiel",talend_tracking
17,NaN,tPostgresqlInput,"SELECT \n talend_tracking.public.job_referentiel.projet, \n talend_tracking.public.job_referentiel.nom_fichier, \n talend_tracking.public.job_referentiel.nom_fichier_sans_extension, \n talend_tracking.public.job_referentiel.id_job, \n talend_tracking.public.job_referentiel.label_job, \n talend_tracking.public.job_referentiel.display_job,\r\n talend_tracking.public.job_referentiel.version\nFROM talend_tracking.public.job_referentiel",talend_tracking
18,NaN,tPostgresqlInput,"select id_job,\r\nmax(version) as latest_version\r\nfrom job_referentiel\r\ngroup by id_job",job_referentiel
19,NaN,tPostgresqlInput,"SELECT \n talend_tracking.public.job_referentiel.projet, \n talend_tracking.public.job_referentiel.nom_fichier, \n talend_tracking.public.job_referentiel.nom_fichier_sans_extension, \n talend_tracking.public.job_referentiel.id_job, \n talend_tracking.public.job_referentiel.label_job, \n talend_tracking.public.job_referentiel.display_job,\r\n talend_tracking.public.job_referentiel.version\nFROM talend_tracking.public.job_referentiel",talend_tracking
20,NaN,tPostgresqlInput,"SELECT \n talend_tracking.public.job_referentiel.projet, \n talend_tracking.public.job_referentiel.nom_fichier, \n talend_tracking.public.job_referentiel.nom_fichier_sans_extension, \n talend_tracking.public.job_referentiel.id_job, \n talend_tracking.public.job_referentiel.label_job, \n talend_tracking.public.job_referentiel.display_job,\r\n talend_tracking.public.job_referentiel.version\nFROM talend_tracking.public.job_referentiel",talend_tracking
21,NaN,tPostgresqlInput,"SELECT \n talend_tracking.public.job_referentiel.projet, \n talend_tracking.public.job_referentiel.nom_fichier, \n talend_tracking.public.job_referentiel.nom_fichier_sans_extension, \n talend_tracking.public.job_referentiel.id_job, \n talend_tracking.public.job_referentiel.label_job, \n talend_tracking.public.job_referentiel.display_job,\r\n talend_tracking.public.job_referentiel.version\nFROM talend_tracking.public.job_referentiel",talend_tracking
22,NaN,tPostgresqlInput,"SELECT \n talend_tracking.public.job_referentiel.projet, \n talend_tracking.public.job_referentiel.nom_fichier, \n talend_tracking.public.job_referentiel.nom_fichier_sans_extension, \n talend_tracking.public.job_referentiel.id_job, \n talend_tracking.public.job_referentiel.label_job, \n talend_tracking.public.job_referentiel.display_job,\r\n talend_tracking.public.job_referentiel.version\nFROM talend_tracking.public.job_referentiel",talend_tracking
23,NaN,tPostgresqlInput,"SELECT \n +context.talend_tracking_postgres_Schema+.job_referentiel.projet, \n +context.talend_tracking_postgres_Schema+.job_referentiel.nom_fichier, \n +context.talend_tracking_postgres_Schema+.job_referentiel.nom_fichier_sans_extension, \n +context.talend_tracking_postgres_Schema+.job_referentiel.id_job, \n +context.talend_tracking_postgres_Schema+.job_referentiel.label_job, \n +context.talend_tracking_postgres_Schema+.job_referentiel.display_job, \n +context.talend_tracking_postgres_Schema+.job_referentiel.version\nFROM +context.talend_tracking_postgres_Schema+.job_referentiel",


In [15]:
df1=pd.read_csv("sql_requetes.csv", sep=";")

In [16]:
def extraire_table(sql_query):
    # Expression régulière pour extraire les noms de table
    pattern = r'\b(?:FROM|JOIN)\s+(\w+)\b'

    # Recherche des correspondances dans la requête SQL
    table = re.findall(pattern, sql_query, re.IGNORECASE)
    return table 

# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df1['table'] = df1['requete_sql'].apply(extraire_table)

# S'assurer que la colonne 'table' est de type chaîne
df1['table'] = df1['table'].astype(str)

# Supprimer les crochets de la colonne 'table'
df1['table']=df1['table'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df1['table'] = df1['table'].str.replace('\'', '')

# Afficher le DataFrame résultant
df1.head(10)

projet  \
0    ATP   
1    ATP   
2    ATP   
3    ATP   
4    ATP   
5    ATP   
6    ATP   
7    PLE   
8    PLE   
9    PLE   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [17]:
df.to_csv("table_sql.csv")
df1.to_csv("table_sql1.csv")

In [18]:
#test avec une extraction depuis Talend de la table job_sql_table
#erreur car les requetes se mettent sur plusieurs lignes...

df2=pd.read_csv("out.csv", sep=(","))
df2.head()

FileNotFoundError: [Errno 2] No such file or directory: 'out.csv'